In [ ]:
import os 
import sys
m_path = os.getcwd()
sys.path.append(m_path)
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from stream_functions import StreamFunctions
from boundary_conditions import BoundaryConditions
from temporal_stability import TemporalStability
from two_equation_solver import *
from claude1 import *

sf = StreamFunctions(n_wakes=2)
bc = BoundaryConditions(sf)
D  = bc.M_non_dim
A = bc.A_non_dim
B = bc.B_non_dim

val_set = {'S': 0.2, 'Lambda': -3.1, 'L': 2.0, 'delta': 0.1, 's':-1}
A_sub = A.subs(val_set)
B_sub = B.subs(val_set)

ts = TemporalStability(sf, bc, val_set)
max_threshold = 1e6
alpha_r = np.arange(0,4.01,0.1)
alpha_i = np.arange(-4,4,0.1)
alpha_R, alpha_I = np.meshgrid(alpha_r, alpha_i)
alpha = alpha_R + 1j * alpha_I
w_vals_imag = np.zeros_like(alpha_I)
w_vals = np.zeros_like(alpha)

for i in range(alpha.shape[0]):
    print(f'Row {i+1} of {alpha.shape[0]}')
    for j in range(alpha.shape[1]):
        alpha_val = alpha[i,j]
        c_val_list = ts.find_c(alpha_val)
        temp_w_vals = c_val_list * alpha_val
        temp_w_vals[np.isnan(np.abs(temp_w_vals))] = -1e10-1j*1e10
        temp_w_vals[np.isinf(np.abs(temp_w_vals))] = -1e10-1j*1e10
        temp_w_vals[np.abs(temp_w_vals) > max_threshold] = -1e10-1j*1e10
        temp_w_vals = temp_w_vals[temp_w_vals != 0 + 0j]
        if len(temp_w_vals) == 0:
            temp_max_w_val_imag = np.array([0.0])
        else:
            temp_max_w_val_imag = np.max(np.imag(temp_w_vals)) 
        max_w_val_idx = np.argmax(np.imag(temp_w_vals))
        w_vals[i,j] = temp_w_vals[max_w_val_idx]
        w_vals_imag[i,j] = np.imag(w_vals[i,j])

sio.savemat('C:\\Users\\tvivek\\Desktop\\w_vals_imag.mat', {'w_vals': w_vals, 'alpha_R': alpha_R, 'alpha_I': alpha_I})

In [ ]:
import os
import sys
m_path = os.getcwd()
sys.path.append(m_path)
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from stream_functions import StreamFunctions
from boundary_conditions import BoundaryConditions
from temporal_stability import TemporalStability
from two_equation_solver import *
from claude1 import *

sf = StreamFunctions(n_wakes=2)
bc = BoundaryConditions(sf)
D  = bc.M_non_dim
A = bc.A_non_dim
B = bc.B_non_dim

max_threshold = 1e6
L_vals = np.linspace(1,5,40)
# sinuous case first considered
for L_val in L_vals:
    print(f'L_val = {L_val}')
    val_set = {'S': 0.38, 'Lambda': -3.1, 'L': L_val, 'delta': 0.1, 's':1}
    A_sub = A.subs(val_set)
    B_sub = B.subs(val_set)

    ts = TemporalStability(sf, bc, val_set)
    alpha_r = np.arange(0,4.01,0.25)
    alpha_i = np.arange(-4,4,0.25)
    alpha_R, alpha_I = np.meshgrid(alpha_r, alpha_i)
    alpha = alpha_R + 1j * alpha_I
    w_vals_imag = np.zeros_like(alpha_I)
    w_vals = np.zeros_like(alpha)

    for i in range(alpha.shape[0]):
        for j in range(alpha.shape[1]):
            alpha_val = alpha[i,j]
            c_val_list = ts.find_c(alpha_val)
            temp_w_vals = c_val_list * alpha_val
            temp_w_vals[np.isnan(np.abs(temp_w_vals))] = -1e10-1j*1e10
            temp_w_vals[np.isinf(np.abs(temp_w_vals))] = -1e10-1j*1e10
            temp_w_vals[np.abs(temp_w_vals) > max_threshold] = -1e10-1j*1e10
            temp_w_vals = temp_w_vals[temp_w_vals != 0 + 0j]
            if len(temp_w_vals) == 0:
                temp_max_w_val_imag = np.array([0.0])
            else:
                temp_max_w_val_imag = np.max(np.imag(temp_w_vals))
            max_w_val_idx = np.argmax(np.imag(temp_w_vals))
            w_vals[i,j] = temp_w_vals[max_w_val_idx]
            w_vals_imag[i,j] = np.imag(w_vals[i,j])
            output_path = f'/work/home/vivek/008_ONR_2025/202505200144/data/out.sin/w_vals_imag_Lval_{L_val}.mat'
            sio.savemat(output_path, {'w_vals': w_vals, 'alpha_R': alpha_R, 'alpha_I': alpha_I, 'L_val': L_val})